# Models Evaluation

1. Create models
2. Compare models
3. Export .csv file with results

In [8]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier      # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics                          #Import scikit-learn metrics module for accuracy calculation

In [9]:
train_data = pd.read_pickle("train_data.pkl")
test_data = pd.read_pickle("test_data.pkl")

train_data.describe()

,loan_id,account_id,date_loan,amount_loan,duration,payments,status
count,328.000000,328.000000,328.000000,328.000000,328.000000,328.000000,328.000000
mean,6205.658537,5982.085366,949989.125000,145308.621951,35.853659,4150.932927,0.719512
std,667.985675,3213.262492,9495.504646,105247.318098,16.734752,2193.620989,0.695541
min,4959.000000,2.000000,930705.000000,4980.000000,12.000000,319.000000,-1.000000
25%,5604.250000,3079.000000,940809.250000,68328.000000,24.000000,2368.750000,1.000000
50%,6227.500000,6032.000000,950565.500000,114804.000000,36.000000,3878.500000,1.000000
75%,6737.250000,8564.500000,960525.250000,198600.000000,48.000000,5907.750000,1.000000
max,7308.000000,11362.000000,961227.000000,538500.000000,60.000000,9689.000000,1.000000


In [10]:
col_names = list(train_data.columns)
col_names.remove('status')

X = train_data[col_names]
y = train_data.status


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=1) # 70% training and 30% test


In [12]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [13]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7676767676767676


**ToDo** : Criar ficheiro .csv para submissão no kaggle

In [14]:
y_pred

array([ 1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,
        1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1,  1])